# Analisis de texto en Python: Preprocesamiento

* * * 

<div class="alert alert-success">  
    
### Objetivos de aprendizaje 
    
* Aprender pasos comunes para preprocesamiento de datos de texto, tan bien como especificar operaciones para preprocesamiento de datos de Twitter.
* Conocer los paquetes NPL comunmente usado y acoplarlos.
* Entender tokenizadores, y como cambiar desde la llegada de los grandes modelos de lenguaje.
</div>

### Icons Used in This Notebook
🔔 **Pregunta**: Una pregunta rapida para ayudar a entender que esta sucediendo.<br>
🥊 **Reto**: Ejercicios interactivos. Trabajaremos a traves de esto en el taller!<br>
⚠️ **Advertencia:** Aviso sobre cosas complicadas o errores comunes.<br>
🎬 **Demo**: Mostrando algo más avanzado: para que sepas para qué se puede usar Python!<br> 

### Secciones
1. [Preprocessing](#section1)
2. [Tokenization](#section2)

En esta seccion de tres parte del taller, aprenderemos la construccion de bloques para ejecutar anlisis de texto en Python. Estas tecnicas se encuentran en el dominio de Procesamiento de Lenguaje Natural (NLP). NLP es un campo que se ocupa con identificacion y extrayendo patrones de lenguaje, ante todo escribiendo textos. A lo largo de la serie de talleres, interactuaremos con paquetes para ejecutar analisis de texto: empezando desde metodos de cadena simple hasta paquetes especificos  NLP, tal como `nltk`, `spaCy`, y mas recientes sobre Modelos de Lenguaje Grande (`BERT`).

Ahora, Instalemos estos paquetes correctamente ante de introducirnos en la materia.

In [44]:
# Uncomment the following lines to install packages/model
# %pip install NLTK
# %pip install transformers
# %pip install spaCy
# !python -m spacy download en_core_web_sm

<a id='section1'></a>

# Preprocesamiento

En la parte 1 de este taller, Direccionaremos el primer paso de analisis de texto. Nuestra meta es convertir la fila, datos de texto desordenados en un formato consistente. Este proceso es a menudo llamado **preprocesamiento**, **limpieza de texto**, o **normalización de texto**.

Notaras que al final del procesamiento,nuestro sto es aun en formato que nosostros podemos leer y entender. En la parte 2 y 3, comenzaremos nuestra incursión en la conversión de datos de texto en una representación numérica, un formato que las computadoras pueden manejar más fácilmente. 

🔔 **Pregunta**: Vamos a pausar un momento para reflexionar sobre **sus** previas experiencias trabajando sobre texto de datos. 
- Cual es el formato de los datos de texto con los que has interactuado (plain text, CSV, or XML)?
- De donde viene (structured corpus, scraped from the web, survey data)?
- Esta desordenado (i.e., is the data formatted consistently)?

## Procesos Comunes

Preprocesamiento no es aldo que podamos lograr con una simple linea de código. Nosotros a menudo empezamos por familiarizarnos nosotros mismo con los datos, y en el camino, obtenemos una comprensión más clara de la granularidad del preprocesamiento que queremos aplicar.

Inicialmente, comenzamos aplicando un conjunto de procesos comúnmente utilizados para limpiar los datos. Estas operaciones so alteran facilmente la forma o el significado de los datos; sirven como un procedimiento estandarizado para remodelar los datos en un formato consistente.

La siguientes procesos, por ejemplo, son comunmente aplicados para procesos de textos en ingles de varios generos. Estas operaciones pueden  estar siendo usadas para funciones integrles en Python, tal como metodos`string`, y expresiones regulares. 
- El texto en minuscula
- Remover puntuaciones marcadas 
- Remover caracteres de espacio en blanco
- Remover palabras en stop 

Después al iniciar el procesamiento, nosotros debemos cambiar para realizar procesos de tareas especificas, cuyos detalles a menudo dependen de la tarea posterior que queremos realizar y de la naturaleza de los datos de texto (i.e., its stylistic and linguistic features).  

Antes de adentrarnos en estas operaciones, echemos un vistazo a nuestros datos!


### Importar el texto de datos

El texto de datos, podemos estar trabajando con un archivo CSV. Contiene tuits sobre aerolíneas estadounidenses, eliminados desde febrero de 2015. 

Vamos a leer el archivo `airline_tweets.csv` dentro del dataframe con `pandas`.

In [45]:
!pip show pandas

Name: pandas
Version: 2.2.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License

Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
All rights reserved.

Copyright (c) 2011-2023, Open source contributors.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of the copyright holder nor the names of its
  contributors may be u

In [46]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [47]:
# Import pandas
import pandas as pd

# File path to data
csv_path = ('data/airline_tweets.csv')

# Specify the separator
tweets = pd.read_csv(csv_path, sep=',')

In [48]:
# Show the first five rows
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


El marco de datos tiene una fila por tuit. El texto del tuit se muestra en la columna `text`.
- `text` (`str`): el texto del tweet.

Otros metadatos que nos interesan incluyen: 
- `airline_sentiment` (`str`): the sentiment of the tweet, etiquetado como as "neutral," "positive," o "negative."
- `airline` (`str`): the airline that is tweeted about.
- `retweet count` (`int`): como algunos tiempo el tweet fueron retweeteados.

Echemos un vistazo a algunos de los tweets:

In [49]:
%pip install pandas
# Ejemplo de creación de un DataFrame llamado 'tweets'
import pandas as pd
tweets = pd.DataFrame({
    'text': ['tweet 1', 'tweet 2', 'tweet 3']
})
print(tweets['text'].iloc[0])
print(tweets['text'].iloc[1])
print(tweets['text'].iloc[2])
print(tweets.head())

Note: you may need to restart the kernel to use updated packages.
tweet 1
tweet 2
tweet 3
      text
0  tweet 1
1  tweet 2
2  tweet 3


🔔 **Pregunta**: Que has notado? Cuáles son las características estilísticas de los tweets?

### Lowercasing

Mientras reconocasmos que el uso de mayúsculas y minúsculas de una palabra es informativo, a menudo no trabajamos en contextos en los que podamos utilizar adecuadamente esta información.

Mas a menudo, el análisis posterior nosotros realizamos **case-insensitive**. Por ejemplo, en el análisis de frecuencia, queremos tener en cuenta las diversas formas de una misma palabra. Convertir los datos de texto en minúsculas facilita este proceso y simplifica nuestro análisis.

Podemos lograr fácilmente la conversión a minúsculas con el método de cadena [`.lower()`](https://docs.python.org/3/library/stdtypes.html#str.lower); see [documentation](https://docs.python.org/3/library/stdtypes.html#string-methods) para mas funciones útiles.

Vamos aplicar esto en el sigiente ejemplo:

In [50]:
# Verificar cuántas filas tiene el DataFrame / se cambia el código ya que no se define bien 
print("Número de filas en el DataFrame:", tweets.shape[0])

# Verificar si el índice 108 está dentro del rango de las filas
if tweets.shape[0] > 108:
    first_example = tweets['text'].iloc[108]  # Accede al tweet en el índice 108
    print(first_example)
else:
    print("Índice 108 fuera de rango. El DataFrame solo tiene", tweets.shape[0], "filas.")

Número de filas en el DataFrame: 3
Índice 108 fuera de rango. El DataFrame solo tiene 3 filas.


In [51]:
# Ejemplo: asignar un valor a 'first_example'
first_example = "Este es un ejemplo de texto"

# Dado que el codigo anterior no ejecuta ahora el código esta sin errores

# Revisar si todos los caracteres están en minúscula 
print(first_example.islower())  # Devuelve True si todos los caracteres son minúsculas
print(f"{'=' * 50}")

# Convertirlo a minúsculas
print(first_example.lower())  # Convierte a minúsculas
print(f"{'=' * 50}")

# Convertirlo a mayúsculas
print(first_example.upper())  # Convierte a mayúsculas

False
este es un ejemplo de texto
ESTE ES UN EJEMPLO DE TEXTO


### Eliminar caracteres de espacio en blanco adicionales

A veces nosotros podriamos encontrar textos con espacios en blanco extra, such as spaces, tabs, and newline characters, which is particularly common when the text is scrapped from web pages. Before we dive into the details, let's briefly introduce Regular Expressions (regex) and the `re` package. 

Las expresiones regulares son una forma eficaz de buscar patrones de cadenas específicos en corpus grandes. Su curva de aprendizaje es notablemente pronunciada, pero pueden ser muy eficientes una vez que las dominamos. Muchos paquetes de PNL dependen en gran medida de las expresiones regulares internamente. Los evaluadores de expresiones regulares, como [regex101](https://regex101.com), son herramientas utiles are useful tools tanto en la comprensión como en la creación de expresiones regulares.

Nuestro objetivo en este taller no es proporcionar una inmersión profunda (ni siquiera superficial) en las expresiones regulares; en cambio, queremos exponerlo a ellas para que esté mejor preparado para realizar inmersiones profundas en el futuro.

El siguiente ejemplo es un poema de William Wordsworth. Como muchos poemas, el texto puede contener saltos de línea adicionales. (i.e., newline characters, `\n`) que queremos eliminar.

In [52]:
# File path to the poem
text_path = 'data/poem_wordsworth.txt'

# Read the poem in
with open(text_path, 'r') as file:
    text = file.read()
    file.close()

Como puedes ver, el poema es fomateado como ua cedena continua de textos con los saltos de línea se colocan al final de cada línea, lo que dificulta la lectura. 

In [53]:
text

"I wandered lonely as a cloud\n\n\nI wandered lonely as a cloud\nThat floats on high o'er vales and hills,\nWhen all at once I saw a crowd,\nA host, of golden daffodils;\nBeside the lake, beneath the trees,\nFluttering and dancing in the breeze.\n\nContinuous as the stars that shine\nAnd twinkle on the milky way,\nThey stretched in never-ending line\nAlong the margin of a bay:\nTen thousand saw I at a glance,\nTossing their heads in sprightly dance.\n\nThe waves beside them danced; but they\nOut-did the sparkling waves in glee:\nA poet could not but be gay,\nIn such a jocund company:\nI gazed—and gazed—but little thought\nWhat wealth the show to me had brought:\n\nFor oft, when on my couch I lie\nIn vacant or in pensive mood,\nThey flash upon that inward eye\nWhich is the bliss of solitude;\nAnd then my heart with pleasure fills,\nAnd dances with the daffodils."

Una función útil que podemos utilizar para mostrar el poema correctamente es `.splitlines()`. Como sugiere el nombre, divide una secuencia de texto larga en una lista de líneas siempre que haya un carácter de nueva línea.   

In [54]:
# Dividir la cadena única en una lista de líneas
text.splitlines()

['I wandered lonely as a cloud',
 '',
 '',
 'I wandered lonely as a cloud',
 "That floats on high o'er vales and hills,",
 'When all at once I saw a crowd,',
 'A host, of golden daffodils;',
 'Beside the lake, beneath the trees,',
 'Fluttering and dancing in the breeze.',
 '',
 'Continuous as the stars that shine',
 'And twinkle on the milky way,',
 'They stretched in never-ending line',
 'Along the margin of a bay:',
 'Ten thousand saw I at a glance,',
 'Tossing their heads in sprightly dance.',
 '',
 'The waves beside them danced; but they',
 'Out-did the sparkling waves in glee:',
 'A poet could not but be gay,',
 'In such a jocund company:',
 'I gazed—and gazed—but little thought',
 'What wealth the show to me had brought:',
 '',
 'For oft, when on my couch I lie',
 'In vacant or in pensive mood,',
 'They flash upon that inward eye',
 'Which is the bliss of solitude;',
 'And then my heart with pleasure fills,',
 'And dances with the daffodils.']

Vamos a retornar a nuestros datos tweet para un ejemplo.

In [55]:
# Imprimir el segundo ejemplo 
#second_example = tweets['text'][5]
#second_example
%pip install pandas
import pandas as pd
import os

import pandas as pd
import os

# Verificar el directorio actual de trabajo
print(f"Directorio actual de trabajo: {os.getcwd()}")

# Leer el archivo de texto como líneas simples
file_path = 'data/poem_wordsworth.txt'  

# Leer el archivo línea por línea
with open(file_path, 'r') as file:
    lines = file.readlines()

# Imprimir las primeras 5 líneas del archivo para verificar su contenido
print("Primeras 5 líneas del archivo:")
for i in range(min(5, len(lines))):  # Limitar a 5 líneas
    print(lines[i])

tweets = pd.DataFrame(lines, columns=['text'])

# Verificar el contenido cargado
print("\nPrimeras 5 filas del DataFrame:")
print(tweets.head())

# Ahora, si quieres acceder a un ejemplo específico del DataFrame:
second_example = tweets['text'].iloc[5]  # Acceder al segundo ejemplo
print("\nSegundo ejemplo del DataFrame:")
print(second_example)

# Si deseas realizar otras operaciones en los datos, puedes hacerlo ahora, por ejemplo, convertir el texto a minúsculas:
print("\nTexto en minúsculas del segundo ejemplo:")
print(second_example.lower())

Note: you may need to restart the kernel to use updated packages.
Directorio actual de trabajo: /workspaces/TrabajoFinal
Primeras 5 líneas del archivo:
I wandered lonely as a cloud





I wandered lonely as a cloud

That floats on high o'er vales and hills,


Primeras 5 filas del DataFrame:
                                          text
0               I wandered lonely as a cloud\n
1                                           \n
2                                           \n
3               I wandered lonely as a cloud\n
4  That floats on high o'er vales and hills,\n

Segundo ejemplo del DataFrame:
When all at once I saw a crowd,


Texto en minúsculas del segundo ejemplo:
when all at once i saw a crowd,



En este caso, no queremos dividir el tweet en una lista de cadenas. Seguimos esperando una sola cadena de texto, pero queremos eliminar por completo el salto de línea.

El método string `.strip()` elimina eficazmente los espacios en ambos extremos del texto. Sin embargo, no funcionará en nuestro ejemplo, ya que el carácter de nueva línea está en el medio de el string.

In [56]:
# Solo elimina los espacios en blanco en ambos extremos
second_example.strip()

'When all at once I saw a crowd,'

Este es donde la nube regex es realmete útil.

In [57]:
import re

Ahora, con regex, básicamente, la llamamos para que coincida con un patrón identificado en los datos de texto, y queremos realizar algunas operaciones con la parte coincidente: extraerla, reemplazarla con otra cosa o eliminarla por completo. Por lo tanto, el funcionamiento de las expresiones regulares se puede resumir en los siguientes pasos:

- Identificar y escribir los patrones en regex (`r'PATTERN'`)
- Escribir el remplazo de los patrones(`'REPLACEMENT'`)
- Llamar la función específica regex  (e.g., `re.sub()`)

En nuestro ejemplo, el patron que estamos buscando es `\s`, cuál es el nombre corto en expresión regular para cualquier carácter de espacio en blanco (`\n` and `\t` included). También añadimos un cuantificador `+` en el final: `\s+`. Significa que nos gustaría capturar una o más ocurrencias del carácter de espacio en blanco.

In [58]:
# Escribir un patron en regex
blankspace_pattern = r'\s+'

El reemplazo de uno o más espacios en blanco es exactamente un solo espacio, que es el límite canónico de palabras en inglés. Cualquier espacio adicional se reducirá a un solo espacio. 

In [59]:
# Ecribir un remplazo para identificación de patrones 
blankspace_repl = ' '

Por último, pongamos todo junto usando la función [`re.sub()`](https://docs.python.org/3.11/library/re.html#re.sub), Lo que significa que queremos sustituir un patrón por un reemplazo. La función acepta tres argumentos: el patrón, el reemplazo y la cadena a la que queremos aplicar la función.

In [60]:
# Remplazar los espacios en blanco(s) con ' '
#clean_text = re.sub(pattern = blankspace_pattern, 
                    #repl = blankspace_repl, 
                    #string = second_example)
import re  # Asegúrate de importar el módulo 're'

# Definir el patrón para los espacios en blanco y el reemplazo
blankspace_pattern = r'\s+'  # Esto busca uno o más espacios en blanco
blankspace_repl = ' '  # Esto reemplaza con un solo espacio

# Suponiendo que 'second_example' es el texto que quieres limpiar
second_example = "Este es    un  ejemplo   con    espacios   extras."

# Remplazar los espacios en blanco(s) con ' '
clean_text = re.sub(pattern=blankspace_pattern, 
                    repl=blankspace_repl, 
                    string=second_example)

print(clean_text)

Este es un ejemplo con espacios extras.


Ta-da! El carácter de nueva línea ya no está allí.

### Eliminar puntuaciones marcadas 

A veces sólo nos interesa analizar **alphanumeric characters** (i.e., the letters and numbers), en tal caso podríamos querer eliminar los signos de puntuación. 

El modulo `string` contiene una lista predefinida de puntuaciones marcadas predefinidas. Vamos a imprimir esto.

In [61]:
# Cargar una lista predefinida de signos de puntuación
from string import punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


En la práctica, para eliminar estos caracteres de puntuación, podemos simplemente iterar sobre el texto y eliminar los caracteres que se encuentran en la lista, como se muestra una función a continuación `remove_punct`.

In [62]:
def remove_punct(text):
    '''Remove punctuation marks in input text'''
    
    # Select characters not in puncutaion
    no_punct = []
    for char in text:
        if char not in punctuation:
            no_punct.append(char)

    # Join the characters into a string
    text_no_punct = ''.join(no_punct)   
    
    return text_no_punct

Vamos aplicar la función del ejemplo below. 

In [63]:
# Imprimir el tercer ejemplo
#third_example = tweets['text'][20]
#print(third_example)
#print(f"{'=' * 50}")
# Abrir y leer el archivo de texto
with open('data/poem_wordsworth.txt', 'r', encoding='utf-8') as file:
    content = file.readlines()

# Ver las primeras líneas del archivo para verificar su contenido
for line in content[:5]:  # Muestra las primeras 5 líneas
    print(line)

I wandered lonely as a cloud





I wandered lonely as a cloud

That floats on high o'er vales and hills,



Vamos a intentar con otro tweet. Que has notado?

In [64]:
# Imprimimos otro tweet
#print(tweets['text'][100])
#print(f"{'=' * 50}")
# Leer el archivo como texto 
with open('data/poem_wordsworth.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Crear un DataFrame con las líneas leídas
tweets = pd.DataFrame(lines, columns=['text'])

# Imprimir el primer tweet
print(tweets['text'][0])

# Aplicar la función remove_punct() al primer tweet
import string
def remove_punct(text):
    return ''.join([char for char in text if char not in string.punctuation])

print(remove_punct(tweets['text'][0]))

I wandered lonely as a cloud

I wandered lonely as a cloud



Qué tal el siguiente ejemplo?

In [65]:
# Imprimimos el texto con contracción 
contraction_text = "We've got quite a bit of punctuation here, don't we?!? #Python @D-Lab."

# Aplicar las funciones
remove_punct(contraction_text)

'Weve got quite a bit of punctuation here dont we Python DLab'

⚠️ **Advertencia:** en algún caso, nosotros queremos remover la tokenización de puntuaciones marcadas **after** , cual discutiriamos en minutos. Esto nos diria que el orden de preprocesamiento **order** es un asunto de importancia!

## 🥊 Reto 1: Preprocesamiento con multiples pasos 

Entonces ahora hemos aprendido algunas operaciones de preprocesamiento. ¡Combinémoslas en una función! Esta función te resultará útil si trabajas con datos de texto en inglés confusos y quieres preprocesarlos con una sola función.

A continuación se muestra el ejemplo de datos de texto para el desafío 1. Escribe una función para:
- Convertir el texto en minúsculas
- Eliminar signos de puntuación
- Eliminar espacios en blanco adicionales

Puedes reciclar el código que usamos anteriormente.!

In [66]:
challenge1_path = 'data/example1.txt'

with open(challenge1_path, 'r') as file:
    challenge1 = file.read()
    
print(challenge1)



This is a text file that has some extra blankspace at the start and end. Blankspace is a catch-all term for spaces, tabs, newlines, and a bunch of other things that computers distinguish but to us all look like spaces, tabs and newlines.


The Python method called "strip" only catches blankspace at the start and end of a string. But it won't catch it in       the middle,		for example,

in this sentence.		Once again, regular expressions will

help		us    with this.





In [67]:
def clean_text(text):

    # Step 1: Lowercase
    text = ...

    # Step 2: Use remove_punct to remove punctuation marks
    text = ...

    # Step 3: Remove extra whitespace characters
    text = ...

    return text

In [68]:
# Descomentar la aplicación sobre la función del reto 1 
clean_text(challenge1)

Ellipsis

## Procesos de Tareas específicas

Ahora que comprendemos las operaciones comunes de preprocesamiento, aún quedan algunas operaciones adicionales por considerar. Nuestros datos de texto podrían requerir una mayor normalización según el idioma, la fuente y el contenido de los datos.

Por ejemplo, si trabajamos con documentos financieros, podríamos querer estandarizar los símbolos monetarios convirtiéndolos en dígitos. En nuestros datos de tuits, existen numerosos hashtags y URL. Estos pueden reemplazarse con marcadores de posición para simplificar el análisis posterior.s.

### 🎬 **Demo**: Eliminar Hashtags y URLs 

Aunque las URL, los hashtags y los números son informativos por sí mismos, a menudo no nos importa su significado exacto.

Si bien podríamos eliminarlos por completo, suele ser informativo saber que existe una URL o un hashtag. En la práctica, reemplazamos las URL y los hashtags individuales con un "símbolo" que preserva la existencia de estas estructuras en el texto. Lo habitual es usar las cadenas "URL" y "HASHTAG".

Dado que estos tipos de texto suelen seguir una estructura regular, son un ejemplo adecuado para el uso de expresiones regulares. Apliquemos estos patrones a los datos de los tweets.

In [69]:
# Imprimir un ejemplo de tweet 
url_tweet = tweets['text'][13]
print(url_tweet)

Along the margin of a bay:



In [70]:
# URL 
url_pattern = r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])'
url_repl = ' URL '
re.sub(url_pattern, url_repl, url_tweet)

'Along the margin of a bay:\n'

In [71]:
# Hashtag
hashtag_pattern = r'(?:^|\s)[＃#]{1}(\w+)'
hashtag_repl = ' HASHTAG '
re.sub(hashtag_pattern, hashtag_repl, url_tweet)

'Along the margin of a bay:\n'

## Referencias

1. A tutorial introducing the tokenization scheme in BERT: [The huggingface NLP course on wordpiece tokenization](https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt)
2. A specific example of "failure" in tokenization: [Weaknesses of wordpiece tokenization: Findings from the front lines of NLP at VMware.](https://medium.com/@rickbattle/weaknesses-of-wordpiece-tokenization-eb20e37fec99)
3. How does BERT decide boundaries between subtokens: [Subword tokenization in BERT](https://tinkerd.net/blog/machine-learning/bert-tokenization/#subword-tokenization)

<div class="alert alert-success">

## ❗ Puntos claves 

* Preprocesamiento incluido en los ultimos pasos, algunos de estos son mas comunes para datos de textos independientes, y algunas son tareas especificas. 
* Ambas `nltk` y `spaCy` podría utilizarse para tokenizar y eliminar palabras vacías. Esta última opción es más eficaz para proporcionar diversas anotaciones lingüísticas. 
* La tokenización funciona de manera diferente en BERT, que a menudo implica dividir una palabra completa en subpalabras. 

</div>